In [20]:
import requests
import bs4
from bs4 import BeautifulSoup as soup
import json
import pandas

import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
from collections import defaultdict
nlp = spacy.load('en_core_web_md')
from string import punctuation

In [21]:
def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    page = soup(req.text, parser)
    return json.loads("".join(page.find("script", {"type":"application/ld+json"}).contents))

In [46]:
url1 = "https://www.allrecipes.com/recipe/232550"

In [47]:
json_ld = get_ld_json(url1)
print(json_ld)

[{'@context': 'http://schema.org', '@type': 'BreadcrumbList', 'itemListElement': [{'@type': 'ListItem', 'position': 1, 'item': {'@id': 'https://www.allrecipes.com/', 'name': 'Home', 'image': None}}, {'@type': 'ListItem', 'position': 2, 'item': {'@id': 'https://www.allrecipes.com/recipes/', 'name': 'Recipes', 'image': None}}, {'@type': 'ListItem', 'position': 3, 'item': {'@id': 'https://www.allrecipes.com/recipes/78/breakfast-and-brunch/', 'name': 'Breakfast and Brunch Recipes', 'image': None}}, {'@type': 'ListItem', 'position': 4, 'item': {'@id': 'https://www.allrecipes.com/recipes/1304/breakfast-and-brunch/breakfast-bread/', 'name': 'Breakfast Bread Recipes', 'image': None}}]}, {'@context': 'http://schema.org', '@type': 'Recipe', 'mainEntityOfPage': 'https://www.allrecipes.com/recipe/232550/pumpkin-sweet-potato-bread/', 'name': 'Pumpkin Sweet Potato Bread', 'image': {'@type': 'ImageObject', 'url': 'https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.media-allrecipes.

In [61]:
text = json_ld[1]['recipeInstructions']
if len(text) > 1:
    new_text = ''
    for i,t in enumerate(text):
#         print(text[i]['text'])
        new_text = new_text + text[i]['text']
    text = new_text
else:
    text = text[0]['text']
print(text)

Preheat oven to 325 degrees F (165 degrees C). Grease 2 loaf pans.
Mix pumpkin pie filling, sweet potato pie filling, water, and oil together in a bowl; add flour, baking soda, and baking powder. Stir well. Pour mixture into the prepared loaf pans.
Bake in the preheated oven until a knife inserted into each loaf comes out clean, about 1 hour.



In [74]:
doc = nlp('Preheat oven to 325 degrees F (165 degrees C). Grease 2 loaf pans')
for token in doc:
    print(token.text,token.pos_, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
displacy.render(doc, style="dep")

Preheat PROPN nsubj oven VERB []
oven VERB ROOT oven VERB [Preheat, to, .]
to ADP prep oven VERB [F]
325 NUM nummod degrees NOUN []
degrees NOUN compound F NOUN [325]
F NOUN pobj to ADP [degrees, C, )]
( PUNCT punct C NOUN []
165 NUM nummod degrees NOUN []
degrees NOUN compound C NOUN [165]
C NOUN appos F NOUN [(, degrees]
) PUNCT punct F NOUN []
. PUNCT punct oven VERB []
Grease NOUN ROOT Grease NOUN [pans]
2 NUM nummod loaf NOUN []
loaf NOUN compound pans NOUN [2]
pans NOUN dobj Grease NOUN [loaf]


### Thing that could be tools
#### 1) pattern: lemma: 'in',   

In [63]:
def find_tool(sentence):
    pattern = [{"LEMMA":{"IN":['in','with']}},{"POS":'DET'},{"POS":'NOUN', 'OP':'+'}]
    # tool_pattern = [{"POS":'NOUN', 'OP':'+'}]
    tools = []
    matcher = Matcher(nlp.vocab)
    matcher.add('find', [pattern])
    doc = nlp(sentence)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        for token in span:
            if token.pos_ == 'NOUN':
                tools.append(token)
    return tools
find_tool('Heat butter in a pan over medium heat')
        
        
    

[pan]

In [64]:
tools = []
for sentence in nlp(text).sents:
    print(sentence)
    t = find_tool(str(sentence))
    tools.append(t)
print(tools)
    
    
    

Preheat oven to 325 degrees F (165 degrees C).
Grease 2 loaf pans.

Mix pumpkin pie filling, sweet potato pie filling, water, and oil together in a bowl; add flour, baking soda, and baking powder.
Stir well.
Pour mixture into the prepared loaf pans.

Bake in the preheated oven until a knife inserted into each loaf comes out clean, about 1 hour.

[[], [], [bowl], [], [], []]
